Machine Learning - Exercise 7 

Goal of the exercise is to **code** the **Decision tree algorithm** which is focused on the optimum split part using either **gini index** or **entrophy**.

**Scikit-learn** documentation [Decision Tree](https://scikit-learn.org/stable/modules/tree.html#tree)


In [41]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math

# Load the Iris dataset

https://archive.ics.uci.edu/dataset/53/iris

- One of the earliest datasets used in the literature on classification methods and widely used in statistics and machine learning.
- The data set contains 3 classes of 50 instances each, where each class refers to a type of iris plant.

In [42]:
df = pd.read_csv('https://github.com/lowoncuties/VSB-FEI-Machine-Learning-Exercises/raw/main/datasets/ml_06/iris.csv')
print(df.head())

   sepal_length  sepal_width  petal_length  petal_width species
0           5.1          3.5           1.4          0.2  setosa
1           4.9          3.0           1.4          0.2  setosa
2           4.7          3.2           1.3          0.2  setosa
3           4.6          3.1           1.5          0.2  setosa
4           5.0          3.6           1.4          0.2  setosa


In [43]:
# Visualize the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [44]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

In [45]:
import numpy as np

def gini(y):
    classes = np.unique(y)
    impurity = 1.0
    for c in classes:
        p = np.sum(y == c) / len(y)
        impurity -= p ** 2
    return impurity

In [46]:
def split_dataset(X, y, feature_index, threshold):
    left_mask = X[:, feature_index] <= threshold
    right_mask = X[:, feature_index] > threshold
    return X[left_mask], y[left_mask], X[right_mask], y[right_mask]

In [47]:
def best_split(X,y,criterion='gini'):
    best_feature, best_threshold, best_score = None,None, float('inf')
    n_features = X.shape[1]
    
    for feature_index in range(n_features):
        thresholds = np.unique(X[:, feature_index])
        for threshold in thresholds:
            X_left, y_left, X_right, y_right = split_dataset(X, y, feature_index, threshold)
            if len(y_left) == 0 or len(y_right) == 0:
                continue
            
            if criterion == 'gini':
                score = (len(y_left) * gini(y_left) + len(y_right) * gini(y_right)) / len(y)
            else:
                raise ValueError("Unsupported criterion")
            
            if score < best_score:
                best_score = score
                best_feature = feature_index
                best_threshold = threshold
    return best_feature, best_threshold, best_score

In [48]:
def build_tree(X, y, depth=0, max_depth=3):
    if len(set(y)) == 1 or depth == max_depth:
        return {'leaf': True, 'class': max(set(y), key=list(y).count)}

    feature, threshold, _ = best_split(X, y, criterion='gini')
    if feature is None:
        return {'leaf': True, 'class': max(set(y), key=list(y).count)}

    X_left, y_left, X_right, y_right = split_dataset(X, y, feature, threshold)
    return {
        'leaf': False,
        'feature': feature,
        'threshold': threshold,
        'left': build_tree(X_left, y_left, depth + 1, max_depth),
        'right': build_tree(X_right, y_right, depth + 1, max_depth)
    }


In [49]:
def predict_one(x, tree):
    if tree['leaf']:
        return tree['class']
    if x[tree['feature']] <= tree['threshold']:
        return predict_one(x, tree['left'])
    else:
        return predict_one(x, tree['right'])


In [50]:
tree = build_tree(X, y, max_depth=3)
print(tree)

preds = [predict_one(x, tree) for x in X]
accuracy = np.mean(preds == y)
print("Accuracy:", accuracy)


{'leaf': False, 'feature': 2, 'threshold': np.float64(1.9), 'left': {'leaf': True, 'class': 'setosa'}, 'right': {'leaf': False, 'feature': 3, 'threshold': np.float64(1.7), 'left': {'leaf': False, 'feature': 2, 'threshold': np.float64(4.9), 'left': {'leaf': True, 'class': 'versicolor'}, 'right': {'leaf': True, 'class': 'virginica'}}, 'right': {'leaf': False, 'feature': 2, 'threshold': np.float64(4.8), 'left': {'leaf': True, 'class': 'virginica'}, 'right': {'leaf': True, 'class': 'virginica'}}}}
Accuracy: 0.9733333333333334


In [51]:
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(criterion='gini', max_depth=3)
clf.fit(X, y)
y_pred = clf.predict(X)
print("Sklearn Accuracy:", accuracy_score(y, y_pred))

Sklearn Accuracy: 0.9733333333333334
